In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count, coalesce
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
import time
import warnings
import json
from pyspark.sql import Window
from pyspark.sql.functions import lag, col
import mysql.connector as database
import time

# from mock.tasks import adiciona_carro}
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder \
           .appName('SparkByExamples') \
           .config("spark.jars", "/usr/share/java/mariadb-java-client.jar") \
           .getOrCreate()
sql = SQLContext(ss)

warnings.simplefilter(action='ignore', category=FutureWarning)

session.execute("USE simulacao")

params = json.load(open('./mock/parametros.json'))

connection = database.connect(
    host="host.docker.internal",
    port=3306,
    user="root",
    password="secret"
)

cursor = connection.cursor()
cursor.execute("USE dashboard;")

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
list(session.execute("SELECT MAX(tempo_da_simulacao) FROM simulacao"))

[Row(system_max_tempo_da_simulacao=None)]

In [8]:
list(session.execute("SELECT COUNT(*) FROM simulacao"))[0][0]

0

In [40]:
start_time = time.time()
total = 0
for i in range(10):
    start_time2 = time.time()
    a, b = 100*i,0# 100*i + 100
    query = f"SELECT * FROM simulacao WHERE tempo_da_simulacao >= {a} AND tempo_da_simulacao <= {b} ALLOW FILTERING;"
    r = list(session.execute(query))
    if r != []:
        df = ss.createDataFrame(r)
        
        p = [[key]+list(params[key].values()) for key in params.keys()]
        p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))
        
        Velocidades_Maximas = p.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
        Aceleracoes_Maximas = p.select(F.col('rodovia'), 0.8*F.col("AceleracaoMaxima"))
        Aceleracoes_Maximas = Aceleracoes_Maximas.withColumnRenamed("(AceleracaoMaxima * 0.8)", "AceleracaoMaxima")
        
        collision_tolerance = 1
        collision_tolerance_quad = 0.5
        windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao")
        
        df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
        df = df.withColumn("prev_tempo_da_simulacao", lag("tempo_da_simulacao", 1).over(windowSpec))
        df = df.withColumn("vel_y", (col("pos_y") - col("prev_pos_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))
        df = df.withColumn("prev_vel_y", lag("vel_y", 1).over(windowSpec))
        df = df.withColumn("acel_y", (col("vel_y") - col("prev_vel_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))
        
        df = df.withColumn("posicao_prevista", col("pos_y") + col("vel_y") * (collision_tolerance) + col("acel_y") * collision_tolerance_quad)
        
        window_spec_rf = Window.partitionBy("rodovia", "pos_x").orderBy('pos_y')
        lag_column = col("posicao_prevista") - lag(col("posicao_prevista")).over(window_spec_rf)
        lead_column = lead(col("posicao_prevista")).over(window_spec_rf) - col("posicao_prevista")
        
        # Add the lag column to the DataFrame
        df = df.withColumn("Risco_Colisão", when(((lag_column < 0) & (col("rodovia") == lag(col("rodovia")).over(window_spec_rf)) & (col("pos_x") == lag(col("pos_x")).over(window_spec_rf)))| ((lead_column < 0) & (col("rodovia") == lead(col("rodovia")).over(window_spec_rf)) & (col("pos_x") == lead(col("pos_x")).over(window_spec_rf))), 1).otherwise(0))
        
        df = df.join(Velocidades_Maximas,on='rodovia',how='left')
        df = df.join(Aceleracoes_Maximas,on='rodovia',how='left')
        
        df = df.withColumn('acima_vel',F.abs(col('vel_y'))>F.abs(col('VelocidadeMaxima')))
        df = df.withColumn('acima_acel',F.abs(col('acel_y'))>F.abs(col('AceleracaoMaxima')))
        
        df = df.withColumn("troca_faixa", col("pos_x") != lag("pos_x", 1).over(windowSpec))
        
        # contador de trocas
        
        df = df.withColumn('multado',((F.col('acima_vel') == 1) & (lag('acima_vel').over(windowSpec) == 0)))
        
        windowSpec = Window.partitionBy("placa",'rodovia').orderBy('tempo_da_simulacao')
        
        df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
        df = df.withColumn("on_road", (((col("pos_y") > 0) & (col('pos_y') < 800)) & (col("prev_pos_y") > 0)).cast("int"))
        df = df.withColumn('time_on_road',F.lit(0))
    
        data = df.select('placa','pos_x','pos_y','acel_y','vel_y', 'rodovia', 'tempo_da_simulacao',F.col('multado').cast('int'),'Risco_Colisão',F.col('troca_faixa'))#,'acima_vel')
        datap = data.toPandas()
    
        query = f""" INSERT IGNORE INTO carros (placa, pos_x, pos_y, aceleracao, velocidade, rodovia, horario_registro, multas, risco_colisao, direcao_perigosa)
    
                    VALUES {','.join([str(i) for i in list(datap.to_records(index=False))])};
                    
                    """.replace("None", "NULL").replace("\n", "").replace("nan", "NULL")
    
        cursor.execute(query)
        connection.commit()
    
    
    end_time = time.time()
    elapsed_time = end_time - start_time2
    total = total + elapsed_time
    print(f"For iterarion {i} - Elapsed time: {elapsed_time} seconds")

    cursor.execute("SELECT COUNT(*) FROM carros;")
    print("Número de linhas no DB:", cursor.fetchall())

print()
print(f"Total time for {i+1} iterations: {total}")

For iterarion 0 - Elapsed time: 0.004286050796508789 seconds
Número de linhas no DB: [(975,)]
For iterarion 1 - Elapsed time: 0.004055023193359375 seconds
Número de linhas no DB: [(975,)]
For iterarion 2 - Elapsed time: 0.0038137435913085938 seconds
Número de linhas no DB: [(975,)]
For iterarion 3 - Elapsed time: 0.0035429000854492188 seconds
Número de linhas no DB: [(975,)]
For iterarion 4 - Elapsed time: 0.003024578094482422 seconds
Número de linhas no DB: [(975,)]
For iterarion 5 - Elapsed time: 0.002360820770263672 seconds
Número de linhas no DB: [(975,)]
For iterarion 6 - Elapsed time: 0.0024404525756835938 seconds
Número de linhas no DB: [(975,)]
For iterarion 7 - Elapsed time: 0.0020813941955566406 seconds
Número de linhas no DB: [(975,)]
For iterarion 8 - Elapsed time: 0.002425670623779297 seconds
Número de linhas no DB: [(975,)]
For iterarion 9 - Elapsed time: 0.003019094467163086 seconds
Número de linhas no DB: [(975,)]

Total time for 10 iterations: 0.031049728393554688


In [ ]:
cursor.execute("SELECT COUNT(*) FROM carros;")

In [35]:
cursor.fetchall()

[(975,)]

In [36]:
cursor.execute("SELECT * FROM carros;")

In [38]:
for j in cursor.fetchall():
    print(j)

('PER1H23', 10, 'BR-116', 385.0, 86.35, None, None, None, 0, None)
('CHI0G12', 13, 'BR-116', 745.0, 860.0, None, None, None, 0, None)
('PER1H23', 13, 'BR-116', 385.0, 105.3, None, 6.317, 0, 0, 0)
('CHI0G12', 15, 'BR-116', 745.0, 855.05, None, -2.475, 0, 0, 0)
('ARG1N23', 16, 'BR-116', 475.0, 52.0, None, None, None, 0, None)
('CHI0G12', 16, 'BR-116', 745.0, 850.15, -2.425, -4.9, 0, 0, 0)
('PER1H23', 16, 'BR-116', 385.0, 115.0, -1.028, 3.233, 0, 0, 0)
('ARG1N23', 17, 'BR-116', 475.0, 54.0, None, 2.0, 0, 0, 0)
('CHI0G12', 17, 'BR-116', 745.0, 845.3, 0.05, -4.85, 0, 0, 0)
('PER1H23', 17, 'BR-116', 385.0, 124.85, 6.617, 9.85, 1, 0, 0)
('ARG1N23', 18, 'BR-116', 475.0, 56.1, 0.1, 2.1, 0, 0, 0)
('CHI0G12', 18, 'BR-116', 745.0, 840.5, 0.05, -4.8, 0, 1, 0)
('PER1H23', 18, 'BR-116', 385.0, 134.85, 0.15, 10.0, 0, 0, 0)
('ARG1N23', 19, 'BR-116', 475.0, 58.35, 0.15, 2.25, 0, 0, 0)
('CHI0G12', 19, 'BR-116', 745.0, 835.75, 0.05, -4.75, 0, 0, 0)
('PER1H23', 19, 'BR-116', 385.0, 145.0, 0.15, 10.15, 0, 0